In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('agg-config.txt')


time_step = int(config.get('Configuration', 'time_step'))
time_step2 = int(config.get('Configuration', 'time_step2'))

window_size = int(config.get('Configuration', 'window_size'))
processed_data_folder = config.get('Configuration', 'processed_data_folder')

ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')



activities=config.get('Configuration', 'activities')
activities = [(item.strip()) for item in activities.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))


activity_data = {}

for activity in activities:
    try:
        cut0 = float(config.get(activity, 'cut0'))
    except:
        cut0=0.5
    try:
        user_location = config.get(activity, 'user_location').split(':')
        agg_location = user_location[0]
        ftw_location=config.get(activity, user_location[1])
        ftw_location = [float(item.strip()) for item in ftw_location.split(',')]
    except:
        agg_location="mean"
        ftw_location=[1,0.5]        
    try:
        object_sensors=config.get(activity, 'object_sensors')
        object_sensors= [(item.strip()) for item in object_sensors.split(',')]
        
        processing=config.get(activity, 'processing')
        processing= [(item.strip()) for item in processing.split(',')]
        #condition = condition.replace('&', ' and ').replace('|', ' or ')
    except:
        object_sensors=[]
        processing=None
        print("none object_sensors")
    
    activity_data[activity] = {
        'cut0': cut0,
        'agg_location' : agg_location,
        'ftw_location' : ftw_location,
        'object_sensors': object_sensors,
        'processing': processing
    }

for activity, data in activity_data.items():
    print(f"Activity: {activity}")
    print(f"data: {data}")




none object_sensors
none object_sensors
none object_sensors
none object_sensors
Activity: cooking
data: {'cut0': 0.25, 'agg_location': 'mean', 'ftw_location': [1.0, 1.0, 0.5], 'object_sensors': ['micro_5', 'pans_8', 'dishes_9', 'fridge_13', 'tap_22'], 'processing': ['sensor_kitchen', 'sensor_kitchen2', 'sensor_kitchen3', 'sensor_kitchen_tA', 'cooking']}
Activity: shower
data: {'cut0': 0.2, 'agg_location': 'mean', 'ftw_location': [1, 0.5], 'object_sensors': ['shower_24_apertura', 'hum_shower_25', 'shower_31'], 'processing': ['sensor_shower', 'shower']}
Activity: toileting
data: {'cut0': 0.25, 'agg_location': 'max', 'ftw_location': [1.0, 1.0, 1.0, 0.5], 'object_sensors': ['wc_17', 'sink_21'], 'processing': ['wc_tA', 'sink_tA', 'toileting']}
Activity: pc
data: {'cut0': 0.25, 'agg_location': 'mean', 'ftw_location': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.6, 0.4, 0.2], 'object_sensors': ['pc_1_current_consumption'], 'processing': ['pc']}
Activity: sleep
data: {'cut0': 0.25

In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime, timedelta

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step2)

def sizeT():
    return relT(tN) 

def absT(tt):
    return (int)(tt*time_step2+t0)

day=list(range(day_time(t0),day_time(tN)+1))[0]

print(day)
nT=(int)(round(((tN)-(t0))/time_step2))
print(nT)

from datetime import datetime, timedelta


def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")




day_str=getStrDatefrom(day)
print(day_str)
users=config.get('Configuration', day_str)
users = [(item.strip()) for item in users.split(',')]


print(users)

2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59
2024-07-31 02:00:00
2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00
19935
1440
2024-07-31
['16fe', '5b66', 'ed9c']


In [3]:
import os
from PIL import Image 
import cv2

def getGTImg(tt,user):
    file_path = os.path.join(processed_data_folder+"DAY_"+str(day)+"/"+user+"_agg/FP."+str(tt)+".png")
    if os.path.exists(file_path):
        return rescale_image(np.array(Image.open(file_path)),8) / 255.0
    else:
        return None
    
def getAREAactivity(activity,user):
    file_path = os.path.join(processed_data_folder+"/mask_loc_act/"+str(activity)+".png")
    if os.path.exists(file_path):
        return np.array(Image.open(file_path)) / 255.0
    else:
        file_path = os.path.join(processed_data_folder+"/mask_loc_act/"+str(user)+"."+activity+".png")
        if os.path.exists(file_path):
            return np.array(Image.open(file_path)) / 255.0
        print("ERROR MASK ACTIVITY",activity,user)
        return None
    
def rescale_image(image, scale_factor):
    width = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    dimensions = (width, height)
    rescaled_image = cv2.resize(image, dimensions, interpolation=cv2.INTER_LINEAR)
    return rescaled_image

In [4]:
def intervals(s,val0):
    ret = []
    lastV = 0
    lastI = -1
    for t in range(0, len(s)):
        if lastV >= val0 and s[t] < val0:
            #print("Interval end at index",t,"degree: ", s[t])
            ret.append((lastI, t - 1))
        if lastV < val0 and s[t] >= val0:
            #print("Interval start at index",t,"degree: ", s[t])
            lastI = t
        lastV = s[t]
    
    if lastV >= val0:
        ret.append((lastI, len(s) - 1))
    print(ret)
    return ret

In [5]:

def FTW_f(s,ftw_f):
    s2=np.zeros(len(s))
    for t in range(0,nT):
        s2[t]=0
        for incT in range(0, len(ftw_f)):
            if(t+incT>=(len(s))):
                continue
            s2[t]=max(s2[t],min(s[t+incT],ftw_f[incT]))
    return s2
def FTW_p(s,ftw_f):
    s2=np.zeros(len(s))
    for t in range(0,nT):
        s2[t]=0
        for incT in range(0, len(ftw_f)):
            if(t-incT<0):
                continue
            s2[t]=max(s2[t],min(s[t-incT],ftw_f[incT]))
    return s2

def FTW_fp(s,ftw_f):
    sp=FTW_p(s,ftw_f)
    sf=FTW_f(s,ftw_f)
    return nonZero(sp,sf)

def FTW_fp2(s,ftw_f):
    sp=FTW_p(s,ftw_f)
    sf=FTW_f(s,ftw_f)
    return nonZeroMax(sp,sf)

def FTW_fw(s,ftw_f):
    s2=np.zeros(len(s))
    for t in range(0,nT):
        s2[t]=0
        wt=0
        for incT in range(0, len(ftw_f)):
            if(t+incT>=(len(s))):
                continue
            s2[t]=s2[t]+s[t+incT]*ftw_f[incT]
            wt=wt+ftw_f[incT]
        s2[t]=s2[t]/wt
    return s2
def FTW_pw(s,ftw_f):
    s2=np.zeros(len(s))
    for t in range(0,nT):
        s2[t]=0
        wt=0
        for incT in range(0, len(ftw_f)):
            if(t-incT<0):
                continue
            s2[t]=s2[t]+s[t-incT]*ftw_f[incT]
            wt=wt+ftw_f[incT]
        s2[t]=s2[t]/wt            
    return s2

def FTW_fpw(s,ftw_f):
    sp=FTW_pw(s,ftw_f)
    sf=FTW_fw(s,ftw_f)
    return nonZero(sp,sf)
    

def nonZero(sp,sf):
    s2=np.zeros(len(sp))
    for t in range(0,nT):
        if(sp[t]==0 and sf[t]==0):
            s2[t]=0
        else:
            s2[t]=min(sp[t],sf[t])
    return s2
def nonZeroMax(sp,sf):
    s2=np.zeros(len(sp))
    for t in range(0,nT):
        if(sp[t]==0 and sf[t]==0):
            s2[t]=0
        else:
            s2[t]=max(sp[t],sf[t])
    return s2


In [6]:
from pyparsing import Word, alphas, alphanums, oneOf, infixNotation, opAssoc

# Definir identificadores (operandos)
operand = Word(alphas + alphanums + '_')

# Definir operadores
operator = oneOf("< > + * #")

# Definir la gramática de expresiones
expr = infixNotation(operand,
                     [
                         ("<", 2, opAssoc.LEFT),
                         (">", 2, opAssoc.LEFT),
                         ("+", 2, opAssoc.LEFT),
                         ("*", 2, opAssoc.LEFT),
                         ("#", 2, opAssoc.LEFT),
                     ])

# Función para parsear y mostrar el resultado
def parse_expression(expression):
    try:
        parsed = expr.parseString(expression, parseAll=True)
        return parsed.asList()
    except Exception as e:
        print(f"Error al parsear la expresión: {e}")
        return None

    



In [7]:
def evaluate(activity,stream_obj,parsed_expr):
    print("Evaluating ",activity,parsed_expr)
    if isinstance(parsed_expr, list):
        if len(parsed_expr) == 3:  # Es una expresión binaria
            left = evaluate(activity,stream_obj,parsed_expr[0])
            op = parsed_expr[1]
            right = evaluate(activity,stream_obj,parsed_expr[2])
            print("\t",left," op:",op,right)
            
            target=f"({left} {op} {right})"
            if (op == "<"):
                ftw_f1 = config.get(activity, right)
                ftw_f1 = [float(item.strip()) for item in ftw_f1.split(',')]
                stream_obj[target] = FTW_p(stream_obj[left], ftw_f1)
            if (op == ">"):
                ftw_f1 = config.get(activity, right)
                ftw_f1 = [float(item.strip()) for item in ftw_f1.split(',')]
                stream_obj[target] = FTW_f(stream_obj[left], ftw_f1)
            if (op == "#"):
                stream_obj[target] = nonZero(stream_obj[left], stream_obj[right])
            if (op == "*"):
                stream_obj[target] = np.minimum(stream_obj[left], stream_obj[right])
            if (op == "+"):
                stream_obj[target] = np.maximum(stream_obj[left], stream_obj[right])
                
            
            
            return f"({left} {op} {right})"  # Aquí puedes implementar tu lógica de evaluación personalizada
        else:
            return parsed_expr
    else:
        return parsed_expr




print(activities)
os.makedirs(processed_data_folder+"DAY_"+str(day)+"/act/", exist_ok=True)
S={}
U={}
for activity in activities:
    print("\t"+activity)
    stream_obj={}
    
    
    processing=activity_data[activity]['processing']
    U[activity]= np.zeros((len(users), nT))
    S[activity]= np.zeros((nT))
    if(processing is None):
        print("\t\t None processing, none filter sensors!")
        stream_obj[activity]=np.ones((nT))
        evaluated_expression=activity
    else:
        S[activity]= np.zeros((nT))
        objs=activity_data[activity]['object_sensors']
        print(objs)
        for obj in objs:
            print("\t\t"+obj)
            series=read_csv(processed_data_folder+"DAY_"+str(day)+"/object_sensor/"+obj+".tsv", sep="\t",parse_dates=True,header=None)
            stream_obj[obj]=  np.array(series.values[:,[1]]).flatten()  
            print("Raw stream_obj:",stream_obj[obj].shape)
     #       stream_obj[obj] = FTW_sensor(stream_obj[obj], activity_data[activity]['ftw_f1'])
     #       print("FTW stream_obj:",stream_obj[obj].shape)

        df = pd.DataFrame(stream_obj)

        # Ejemplo de uso
        for term in processing:
            condition=config.get(activity, term)
            print("condition:",condition)
    
            parsed_expression = parse_expression(condition)
            print("parsed_expression:",parsed_expression)
    
            # Ejemplo de uso
            evaluated_expression = evaluate(activity,stream_obj,parsed_expression[0])
            print("evaluated_expression",condition,evaluated_expression)
            stream_obj[term]=stream_obj[evaluated_expression].copy()

    #filtered_df = df.query(condition)
    #print("filtered_df:",filtered_df)
    lastImg=np.zeros((h*8,w*8))
            
    with open(processed_data_folder+"/DAY_"+str(day)+"/act/"+activity+".tsv", "w") as file: 
        file.write("time")
        for ox,obj in enumerate(stream_obj):
            file.write("\t"+obj)
        #file.write("\t"+evaluated_expression+"\t"+activity)
        for user in users:     
            file.write("\t"+user)
        file.write("\n")
            

        for tt in range(t0+time_step2,tN-time_step2,time_step2):
            print(float(tt-t0)/float(tN-t0))
            file.write(time2str(tt))

            ti=relT(tt)
            S[activity][ti]=stream_obj[activity][ti]
            for obj in stream_obj:
                 #print("\t\t\t",user,activity,str(tt),"obj",str(obj),stream_obj[obj][ti])
                file.write("\t"+str("{:,.2f}".format(stream_obj[obj][ti])))
            for ux,user in enumerate(users):
                imgAreaActivity=getAREAactivity(activity,user)
                #print(imgAreaActivity.shape)

    
                imgLocation=getGTImg(tt,user)

                A_min = np.min(imgLocation)
                A_max = np.max(imgLocation)
                if(A_min==A_max):
                     #print("\t zero image!")
                    imgLocation=lastImg
                else:
                    lastImg=imgLocation
                    imgLocation = (imgLocation - A_min) / (A_max - A_min)

                #print(user,tt,A_min,A_max,imgAreaActivity.shape,imgLocation.shape)
                degree = np.max(np.minimum(imgLocation, imgAreaActivity))
                #print(degree)
                timestamp=pd.to_datetime(time2str(tt))
                #print("\t\t\t",user,activity,ti,str(timestamp),"IMG",str(degree))
                file.write("\t"+"{:,.2f}".format(degree))
                U[activity][ux][ti]=degree
            #file.write("\t"+str(stream_obj[obj][ti]))
            
            file.write("\n")
    

    
            

['cooking', 'shower', 'toileting', 'pc', 'sleep', 'kitchen', 'resting', 'exit']
	cooking
['micro_5', 'pans_8', 'dishes_9', 'fridge_13', 'tap_22']
		micro_5
Raw stream_obj: (1440,)
		pans_8
Raw stream_obj: (1440,)
		dishes_9
Raw stream_obj: (1440,)
		fridge_13
Raw stream_obj: (1440,)
		tap_22
Raw stream_obj: (1440,)
condition: (micro_5 + pans_8) + dishes_9
parsed_expression: [[['micro_5', '+', 'pans_8'], '+', 'dishes_9']]
Evaluating  cooking [['micro_5', '+', 'pans_8'], '+', 'dishes_9']
Evaluating  cooking ['micro_5', '+', 'pans_8']
Evaluating  cooking micro_5
Evaluating  cooking pans_8
	 micro_5  op: + pans_8
Evaluating  cooking dishes_9
	 (micro_5 + pans_8)  op: + dishes_9
evaluated_expression (micro_5 + pans_8) + dishes_9 ((micro_5 + pans_8) + dishes_9)
condition: fridge_13+tap_22
parsed_expression: [['fridge_13', '+', 'tap_22']]
Evaluating  cooking ['fridge_13', '+', 'tap_22']
Evaluating  cooking fridge_13
Evaluating  cooking tap_22
	 fridge_13  op: + tap_22
evaluated_expression fri

parsed_expression: [[['sensor_kitchen_tA', '>', 'tA'], '#', ['sensor_kitchen_tA', '<', 'tA']]]
Evaluating  cooking [['sensor_kitchen_tA', '>', 'tA'], '#', ['sensor_kitchen_tA', '<', 'tA']]
Evaluating  cooking ['sensor_kitchen_tA', '>', 'tA']
Evaluating  cooking sensor_kitchen_tA
Evaluating  cooking tA
	 sensor_kitchen_tA  op: > tA
Evaluating  cooking ['sensor_kitchen_tA', '<', 'tA']
Evaluating  cooking sensor_kitchen_tA
Evaluating  cooking tA
	 sensor_kitchen_tA  op: < tA
	 (sensor_kitchen_tA > tA)  op: # (sensor_kitchen_tA < tA)
evaluated_expression (sensor_kitchen_tA > tA)  # (sensor_kitchen_tA < tA) ((sensor_kitchen_tA > tA) # (sensor_kitchen_tA < tA))
0.0006944524820889131
0.0013889049641778262
0.0020833574462667394


0.0027778099283556523
0.0034722624104445653
0.004166714892533479
0.004861167374622392
0.005555619856711305
0.006250072338800218
0.006944524820889131
0.007638977302978044


0.008333429785066957
0.00902788226715587
0.009722334749244783


0.010416787231333696
0.01111123971342261
0.011805692195511521
0.012500144677600435
0.013194597159689347
0.013889049641778261
0.014583502123867175
0.015277954605956087


...

0.9652889501035892
0.965983402585678
0.9666778550677669
0.9673723075498559
0.9680667600319448
0.9687612125140337
0.9694556649961227
0.9701501174782116
0.9708445699603004


0.9715390224423894
0.9722334749244783
0.9729279274065672
0.9736223798886561
0.9743168323707451
0.975011284852834
0.9757057373349228
0.9764001898170118
0.9770946422991007
0.9777890947811896
0.9784835472632785
0.9791779997453675


0.9798724522274563
0.9805669047095452
0.9812613571916342
0.9819558096737231
0.982650262155812
0.9833447146379009
0.9840391671199898
0.9847336196020787
0.9854280720841676
0.9861225245662566
0.9868169770483455
0.9875114295304344


0.9882058820125232
0.9889003344946122
0.9895947869767011
0.99028923945879
0.990983691940879
0.9916781444229679
0.9923725969050567
0.9930670493871457


0.9937615018692346
0.9944559543513235
0.9951504068334124
0.9958448593155014
0.9965393117975903
0.9972337642796791
0.9979282167617681
0.998622669243857


In [8]:
I={}
import copy
U0=copy.deepcopy(U)
for activity in activities:
    cut0=activity_data[activity]['cut0']/2.0
    ftw_location=activity_data[activity]['ftw_location']
    agg_location=activity_data[activity]['agg_location']
    print(activity,cut0,ftw_location)
    I[activity]= np.zeros((len(users), nT))
    for ux,user in enumerate(users):
        print("\t",user)
        print("\t intervals raw ",intervals(S[activity],cut0))
        try:
            FPu=config.get(activity, 'FPu')
            FPu = [float(item.strip()) for item in FPu.split(',')]
            print("FPu",FPu)
            U[activity][ux]=FTW_fp2(U[activity][ux],FPu)
        except:
            print("None FPu")
        #U[activity][ux]=FTW_fp(U[activity][ux],[1,0.5])
        #print("\t interval user:",intervals(I[activity][ux],cut0))
        #I[activity][ux]=FTW_fp(I[activity][ux],ftw_location)
        if(agg_location=='mean' or agg_location is None):
            U[activity][ux][U[activity][ux] <= cut0*4.0] = 0
            I[activity][ux]=np.minimum(U[activity][ux],S[activity])
            I[activity][ux]=FTW_fpw(I[activity][ux],ftw_location)
            I[activity][ux]=FTW_fp(I[activity][ux],ftw_location)
        else:
            if (agg_location=='max'):
                I[activity][ux]=np.minimum(U[activity][ux],S[activity])
                I[activity][ux]=FTW_fp(I[activity][ux],ftw_location)
        print("\t inter intervals:",intervals(I[activity][ux],cut0))
        with open(processed_data_folder+"/DAY_"+str(day)+"/act/"+activity+"."+user+".tsv", "w") as file:    
            for ii in intervals(I[activity][ux],cut0):
                if(agg_location=='mean' or agg_location is None):
                    degree=np.mean(I[activity][ux][ii[0]:ii[1]+1])
                else:
                    if (agg_location=='max'):
                        degree=np.max(I[activity][ux][ii[0]:ii[1]+1])
                    else:
                        print("ERROR DEGREE")
                        degree=-1.0
                print("\t Interval",ii, datetime.fromtimestamp(absT(ii[0])).strftime("%Y-%m-%d %H:%M:%S"), datetime.fromtimestamp(absT(ii[1])).strftime("%Y-%m-%d %H:%M:%S"),degree)
                if degree < cut0:
                    degree=0.0
                file.write(str(ii)+"\t"+datetime.fromtimestamp(absT(ii[0])).strftime("%Y-%m-%d %H:%M:%S")+"\t"+datetime.fromtimestamp(absT(ii[1])).strftime("%Y-%m-%d %H:%M:%S")+"\t")
                file.write("{:,.2f}".format(degree)+"\n")


cooking 0.125 [1.0, 1.0, 0.5]
	 16fe
[(6, 6), (288, 288), (306, 306), (338, 338), (434, 434), (494, 494), (642, 643), (649, 659), (682, 682), (691, 694), (725, 739), (757, 764), (777, 777), (782, 784), (791, 801), (807, 808), (836, 836), (1034, 1034), (1040, 1041), (1070, 1070), (1153, 1154), (1187, 1203), (1211, 1213), (1243, 1244), (1298, 1299), (1327, 1331), (1426, 1427)]
	 intervals raw  [(6, 6), (288, 288), (306, 306), (338, 338), (434, 434), (494, 494), (642, 643), (649, 659), (682, 682), (691, 694), (725, 739), (757, 764), (777, 777), (782, 784), (791, 801), (807, 808), (836, 836), (1034, 1034), (1040, 1041), (1070, 1070), (1153, 1154), (1187, 1203), (1211, 1213), (1243, 1244), (1298, 1299), (1327, 1331), (1426, 1427)]
None FPu
[(306, 306), (338, 338), (762, 764), (777, 777), (782, 784), (791, 791), (1187, 1188), (1298, 1298), (1327, 1327)]
	 inter intervals: [(306, 306), (338, 338), (762, 764), (777, 777), (782, 784), (791, 791), (1187, 1188), (1298, 1298), (1327, 1327)]
[(306,

[(986, 1074)]
	 inter intervals: [(986, 1074)]
[(986, 1074)]
	 Interval (986, 1074) 2024-07-31 18:26:00 2024-07-31 19:54:00 0.8635382242784754
sleep 0.125 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.6, 0.4, 0.2]
	 16fe
[(1, 1438)]
	 intervals raw  [(1, 1438)]
None FPu
[(1, 265), (1333, 1438)]
	 inter intervals: [(1, 265), (1333, 1438)]
[(1, 265), (1333, 1438)]
	 Interval (1, 265) 2024-07-31 02:01:00 2024-07-31 06:25:00 0.7115708019733452
	 Interval (1333, 1438) 2024-08-01 00:13:00 2024-08-01 01:58:00 0.6053644308076095
	 5b66
[(1, 1438)]
	 intervals raw  [(1, 1438)]
None FPu
[(10, 418)]
	 inter intervals: [(10, 418)]
[(10, 418)]
	 Interval (10, 418) 2024-07-31 02:10:00 2024-07-31 08:58:00 0.7917144731653366
	 ed9c
[(1, 1438)]
	 intervals raw  [(1, 1438)]
None FPu
[(1, 287), (799, 964), (1430, 1438)]
	 inter intervals: [(1, 287), (799, 964), (1430, 1438)]
[(1, 287), (799, 964), (

[(338, 338), (563, 568), (763, 790)]
	 inter intervals: [(338, 338), (563, 568), (763, 790)]
[(338, 338), (563, 568), (763, 790)]
	 Interval (338, 338) 2024-07-31 07:38:00 2024-07-31 07:38:00 0.18431372549019606
	 Interval (563, 568) 2024-07-31 11:23:00 2024-07-31 11:28:00 0.20260441872508975
	 Interval (763, 790) 2024-07-31 14:43:00 2024-07-31 15:10:00 0.7013159834698158
	 5b66
[(1, 1438)]
	 intervals raw  [(1, 1438)]
None FPu
[(1, 7), (425, 439), (459, 473), (476, 481), (484, 495), (643, 739), (758, 848), (1039, 1039), (1063, 1088), (1153, 1156), (1174, 1247), (1426, 1438)]
	 inter intervals: [(1, 7), (425, 439), (459, 473), (476, 481), (484, 495), (643, 739), (758, 848), (1039, 1039), (1063, 1088), (1153, 1156), (1174, 1247), (1426, 1438)]
[(1, 7), (425, 439), (459, 473), (476, 481), (484, 495), (643, 739), (758, 848), (1039, 1039), (1063, 1088), (1153, 1156), (1174, 1247), (1426, 1438)]
	 Interval (1, 7) 2024-07-31 02:01:00 2024-07-31 02:07:00 0.451487449499754
	 Interval (425, 439

[(343, 748), (1067, 1183)]
	 inter intervals: [(343, 748), (1067, 1183)]
[(343, 748), (1067, 1183)]
	 Interval (343, 748) 2024-07-31 07:43:00 2024-07-31 14:28:00 0.84038303752645
	 Interval (1067, 1183) 2024-07-31 19:47:00 2024-07-31 21:43:00 0.7910118555279846
	 5b66
[(1, 1438)]
	 intervals raw  [(1, 1438)]
None FPu
[(498, 638), (1094, 1149)]
	 inter intervals: [(498, 638), (1094, 1149)]
[(498, 638), (1094, 1149)]
	 Interval (498, 638) 2024-07-31 10:18:00 2024-07-31 12:38:00 0.8309082589796387
	 Interval (1094, 1149) 2024-07-31 20:14:00 2024-07-31 21:09:00 0.563594470046083
	 ed9c
[(1, 1438)]
	 intervals raw  [(1, 1438)]
None FPu
[(316, 722), (1085, 1418)]
	 inter intervals: [(316, 722), (1085, 1418)]
[(316, 722), (1085, 1418)]
	 Interval (316, 722) 2024-07-31 07:16:00 2024-07-31 14:02:00 0.8469496656623807
	 Interval (1085, 1418) 2024-07-31 20:05:00 2024-08-01 01:38:00 0.8562156938380363


In [9]:
for activity in activities:
    print("#####",activity)
    cut0=activity_data[activity]['cut0']/2.0
    agg_location=activity_data[activity]['agg_location']

    with open(processed_data_folder+"/DAY_"+str(day)+"/act/"+activity+".all.tsv", "w") as file:    
        file.write("index\ttime-ini\ttime-end")
        for ux,user in enumerate(users):
            file.write("\t"+user)
        file.write("\n")
        
        for ux,user in enumerate(users):
            print("##",user)
            for ii in intervals(I[activity][ux],cut0):
                if(agg_location=='mean' or agg_location is None):
                    degree=np.mean(I[activity][ux][ii[0]:ii[1]+1])
                else:
                    if (agg_location=='max'):
                        degree=np.max(I[activity][ux][ii[0]:ii[1]+1])
                    else:
                        print("ERROR DEGREE")
                        degree=-1.0
                print("\t Interval",ii, datetime.fromtimestamp(absT(ii[0])).strftime("%Y-%m-%d %H:%M:%S"), datetime.fromtimestamp(absT(ii[1])).strftime("%Y-%m-%d %H:%M:%S"),degree,I[activity][ux][ii[0]:ii[1]+1],U[activity][ux][ii[0]:ii[1]+1],S[activity][ii[0]:ii[1]+1])
                if degree < cut0:
                    degree=0.0
                file.write(str(ii)+"\t"+datetime.fromtimestamp(absT(ii[0])).strftime("%Y-%m-%d %H:%M:%S")+"\t"+datetime.fromtimestamp(absT(ii[1])).strftime("%Y-%m-%d %H:%M:%S"))
                for ux2,user2 in enumerate(users):
                    if(ux==ux2):
                        file.write("\t"+"{:,.2f}".format(degree))
                    else:
                        file.write("\t0.0")
                file.write("\n")

##### cooking
## 16fe
[(306, 306), (338, 338), (762, 764), (777, 777), (782, 784), (791, 791), (1187, 1188), (1298, 1298), (1327, 1327)]
	 Interval (306, 306) 2024-07-31 07:06:00 2024-07-31 07:06:00 0.39215686274509803 [0.39215686] [0.98039216] [1.]
	 Interval (338, 338) 2024-07-31 07:38:00 2024-07-31 07:38:00 0.26153846153846155 [0.26153846] [0.65384615] [1.]
	 Interval (762, 764) 2024-07-31 14:42:00 2024-07-31 14:44:00 0.49882352941176467 [0.39215686 0.71215686 0.39215686] [0.98039216 0.8        0.98039216] [1. 1. 1.]
	 Interval (777, 777) 2024-07-31 14:57:00 2024-07-31 14:57:00 0.3603305785123967 [0.36033058] [0.90082645] [1.]
	 Interval (782, 784) 2024-07-31 15:02:00 2024-07-31 15:04:00 0.47326797385620917 [0.39215686 0.68882353 0.33882353] [0.98039216 0.875      0.84705882] [1. 1. 1.]
	 Interval (791, 791) 2024-07-31 15:11:00 2024-07-31 15:11:00 0.3689922480620155 [0.36899225] [0.92248062] [1.]
	 Interval (1187, 1188) 2024-07-31 21:47:00 2024-07-31 21:48:00 0.3024614100959533 [0.2

In [10]:
activity="kitchen"
ux=2
cut0=activity_data[activity]['cut0']/2.0
ftw_location=activity_data[activity]['ftw_location']
agg_location=activity_data[activity]['agg_location']


U00=copy.deepcopy(U0)
U0[activity][ux][U0[activity][ux] <= cut0*4.0] = 0
I[activity][ux]=np.minimum(U[activity][ux],S[activity])
I[activity][ux]=FTW_fpw(I[activity][ux],ftw_location)


I[activity][ux]=np.minimum(U0[activity][ux],S[activity])

fp=FTW_fpw(I[activity][ux],ftw_location)
pw=FTW_pw(I[activity][ux],ftw_location)
fw=FTW_fw(I[activity][ux],ftw_location)

for ix,v in enumerate(I[activity][ux]):
    print("\t"+datetime.fromtimestamp(absT(ix)).strftime("%Y-%m-%d %H:%M:%S")+"\t"+str(U00[activity][ux][ix])+"\t"+str(U0[activity][ux][ix])+"\t"+str(v)+"\t"+str(fp[ix])+"\t"+str(pw[ix])+"\t"+str(fw[ix])+"\t"+str(fp[ix]))

	2024-07-31 02:00:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:01:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:02:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:03:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:04:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:05:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:06:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:07:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:08:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:09:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:10:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:11:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:12:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:13:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:14:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:15:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:16:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:17:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:18:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:19:00	0.0	0.0	0.0	0.0	0.0	0.0	0.0
	2024-07-31 02:20:00